In [2]:
import requests
from contextlib import closing
import csv
from plotly.graph_objs import Scattergeo, Layout
from plotly import offline 

# Définition des variables et listes requises
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
confirmed_list = []
lons = []
lats = []
texts = []

# Pour pouvoir lire le url en fichier csv sans utiliser les pandas
with closing(requests.get(url, stream=True)) as r:
    f = (line.decode('utf-8') for line in r.iter_lines())
    reader = csv.reader(f, delimiter=',', quotechar='"')
    for x in reader:
        # le [-1] nous permet d'obtenir les données les plus à jour (la dernière date enregistrée)
        confirmed = x[-1]
        lon = x[3]
        lat = x[2]
        title = x[1]
        provinces = x[0]
        
        # Append chaque item à la liste correspondante
        confirmed_list.append(confirmed)
        lons.append(lon)
        lats.append(lat)
        texts.append(provinces+ '\n' + title + '\n' + confirmed)
        # Pour convertir les items de la liste en int
        liste = [ int(y) for y in confirmed_list[1:]]
        
# Préparation du graphique
graph_data = [{
    'type': 'scattergeo',
    'lon': lons[1:],
    'lat': lats[1:],
    'text': texts[1:],
    'marker': {
        'size': 2*int(confirmed),
        'color': liste,
        'colorscale': 'jet', 
        'colorbar': {'title': 'Nombre de cas'},
    },
}]
# Titre du graphique
graph_layout = Layout(title='Cas Confirmés - COVID-19')

# Génération du graphique, comme fichier externe 'covid_csv.html'
fig = {'data': graph_data, 'layout': graph_layout}

offline.plot(fig, filename='covid_csv.html')

'covid_csv.html'